<a href="https://colab.research.google.com/github/vinhngtr/caro_game_project/blob/master/Mmc_queue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simpy

In [ ]:

# @title Implement M/M/c queue System
import simpy
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from random import seed
import statistics
from tabulate import tabulate
import pandas as pd
from scipy.optimize import curve_fit
class BuffetSystem:
    def __init__(self, env, arrival_rate, service_rate, num_servers, arrival_matrix, leave_matrix):

        self.total_customer = 0
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.num_servers = num_servers
        self.env = env
        self.arrival_matrix = arrival_matrix
        self.leave_matrix = leave_matrix
        temp = self.calculate_arrival_rate()
        self.seafoodQueue = SeafoodQueue(env, temp[0], service_rate[0], num_servers[0], leave_matrix[0])
        self.soupQueue = SoupQueue(env, temp[1], service_rate[1], num_servers[1], leave_matrix[1])
        self.dessertQueue = DessertQueue(env, temp[2], service_rate[2], num_servers[2], leave_matrix[2])

        self.completed_job = 0
        self.current_job = [0]

    def calculate_arrival_rate(self):
        # Define the matrices

        I = np.eye(3)  # 3x3 identity matrix
        P = np.array(self.leave_matrix)  # 3x3 matrix
        a = np.dot(np.array(self.arrival_matrix), self.arrival_rate)  # 1D array representing the vector a

        # Calculate lambda using the provided formula
        PT = np.transpose(P)  # Transpose of P
        inverse_term = np.linalg.inv(I - PT)  # Inverse of (I - P^T)
        lambda_value = np.dot(inverse_term, a)  # Matrix multiplication with a

        # fix small rounding errors may occur when perform calculations involving floating-point numbers
        for i in range(3):
            self.leave_matrix[i][i] = 1 - sum(self.leave_matrix[i])
            self.leave_matrix[i] = self.leave_matrix[i] / np.sum(self.leave_matrix[i])

        # print(leave_matrix)
        return lambda_value

    def run(self, num_customers):
        for i in range(num_customers):
            self.current_job.append(self.current_job[len(self.current_job)-1] + 1)

            next_destination = np.random.choice(len(self.arrival_matrix), p=self.arrival_matrix)
            if next_destination == 0:
                # go to seafood
                self.env.process(self.seafoodQueue.customer_arrival(self, i))
            elif next_destination == 1:
                # go to Soup
                self.env.process(self.soupQueue.customer_arrival(self, i))
            else:
                # go to Dessert
                self.env.process(self.dessertQueue.customer_arrival(self, i))
            t = random.expovariate(self.arrival_rate)
            yield self.env.timeout(t)

    def display_metric(self):
        self.seafoodQueue.end_simulation_time = self.env.now
        self.soupQueue.end_simulation_time = self.env.now
        self.dessertQueue.end_simulation_time = self.env.now
        self.seafoodQueue.get_metrics_table("Seafood")
        self.soupQueue.get_metrics_table("Soup")
        self.dessertQueue.get_metrics_table("Dessert")

        # plotter = PlotMetric(self.seaFoodQueue.number_of_completions, [])
        # plotter.plotNonLinearRegression(self.seaFoodQueue.number_of_completions, "Time", "Throughput", "Non-Linear Regression of Throughput")

class MMNQueue:
    def __init__(self, env, arrival_rate, service_rate, num_servers, leave_list):
        self.env = env
        if num_servers > 0:
            self.server = simpy.Resource(env, capacity=num_servers)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.waiting_times = []
        self.leave_list = leave_list
        self.m = num_servers

        self.total_arrivals = 0
        self.number_of_completions = [[0,0]]
        self.service_times = []
        self.end_simulation_time = 0
        self.response_time = []
        self.simulation_metrics = []
        self.calculation_metrics = []
    def calculate(self, metric_name):
        # Traffic intensity:
        rho = self.arrival_rate / (self.m * self.service_rate)

        sum_term = sum((math.pow(self.m * rho, n) / math.factorial(n) for n in range(1, self.m)))
        # Probability of zero jobs in the system
        P0 = 1 / (1 + (math.pow(self.m * rho, self.m) / (math.factorial(self.m) * (1 - rho))) + sum_term)
        # Probability of all server being busy
        L = (math.pow(self.m * rho, self.m) * P0) / (math.factorial(self.m) * (1 - rho))
        # Mean number of jobs in the queue
        L_q = (rho * L) / (1 - rho)
        # Mean Waiting time
        W = L_q / self.arrival_rate
        # Mean Response time
        R = 1/self.service_rate * (1+ (L/(self.m*(1-rho))))

        if metric_name == "waiting_time":
            return W
        elif metric_name == "response_time":
            return R
        elif metric_name == "traffic_intensity":
            return rho
        return W
    def simulation_waiting_time(self):
        if len(self.waiting_times) > 0:
            return statistics.mean(self.waiting_times)
        return "null"

    def simulation_response_time(self):
        return statistics.mean(self.response_time)

    def simulation_throughput(self):
        return self.number_of_completions[-1][0]/self.number_of_completions[-1][1]

    def simulation_utilization(self):
        return sum(self.service_times)/(self.m*self.end_simulation_time)

    def simulation_service_time(self):
        return statistics.mean(self.service_times)

    def simulation_arrival_time(self):
        return self.end_simulation_time/self.total_arrivals

    def calculate_metrics(self):
        # Calculate base on data
        self.simulation_metrics = []
        self.calculation_metrics = []
        self.simulation_metrics.append(round(self.simulation_response_time(),2))
        self.simulation_metrics.append(round(self.simulation_throughput(),2))
        if len(self.waiting_times) > 0:
            self.simulation_metrics.append(round(self.simulation_waiting_time(),2))
        else:
            self.simulation_metrics.append("null")
        self.simulation_metrics.append(round(self.simulation_utilization(),2))
        self.simulation_metrics.append(round(self.simulation_service_time(),2))
        self.simulation_metrics.append(round(self.simulation_arrival_time(),2))
        # Calculate base on mathematical formula
        self.calculation_metrics.append(round(self.calculate("response_time"),2))
        self.calculation_metrics.append(round(self.arrival_rate,2))
        self.calculation_metrics.append(round(self.calculate("waiting_time"),2))
        self.calculation_metrics.append(round(self.calculate("traffic_intensity"),2))
        self.calculation_metrics.append(round(1/self.service_rate,2))
        self.calculation_metrics.append(round(1/self.arrival_rate,2))

    def get_metrics_table(self, name):
        self.calculate_metrics()
        print("\n\n")
        Utils.print_metric_table(name, self.simulation_metrics, self.calculation_metrics)

class Utils:
    def print_metric_table(name, simulation_data, calculation_data):
        sub = []
        for i in range(len(simulation_data)):
            if simulation_data[i] == "null":
                sub.append("null")
            else:
                sub.append(f'{round((simulation_data[i]-calculation_data[i]),2)}')

        data = {
            'Metric': ['response_time', 'throughput', 'average_waiting_time', 'utilization', "service_time", "interarrival_time"],
            name+ ' Simulation': simulation_data,
            name+ ' Calculation': calculation_data,
            'subtraction': sub
        }

        # Chuyển dữ liệu thành DataFrame
        df = pd.DataFrame(data)

        # In bảng dữ liệu sử dụng tabulate
        table = tabulate(df, headers='keys', tablefmt='pretty', showindex=False)

        # In bảng dữ liệu với đường gạch phân chia
        print(table)
class SeafoodQueue(MMNQueue):
    def customer_arrival(self, buffet_system, customer_id):
        arrival_time = buffet_system.env.now
        with self.server.request() as request:
            yield request
            self.total_arrivals += 1
            service_start_time = buffet_system.env.now
            service_time = random.expovariate(self.service_rate)
            self.service_times.append(service_time)
            yield buffet_system.env.timeout(service_time)
            service_end_time = buffet_system.env.now
            wait_time = service_start_time - arrival_time
            self.waiting_times.append(wait_time)
            self.response_time.append(service_end_time-arrival_time)
            self.number_of_completions.append([self.number_of_completions[-1][0]+1,buffet_system.env.now])
            # calculate next destination
            next_destination = np.random.choice(len(self.leave_list), p=self.leave_list)
            if next_destination == 0:
                # go out
                buffet_system.current_job.append(buffet_system.current_job[len(buffet_system.current_job)-1] - 1)
                buffet_system.completed_job += 1
            elif next_destination == 1:
                # go to Soup
                buffet_system.env.process(buffet_system.soupQueue.customer_arrival(buffet_system, customer_id))
            else:
                # go to Dessert
                a = 3
                buffet_system.env.process(buffet_system.dessertQueue.customer_arrival(buffet_system, customer_id))
class SoupQueue(MMNQueue):
    def customer_arrival(self, buffet_system, customer_id):
        arrival_time = buffet_system.env.now
        with self.server.request() as request:
            yield request
            self.total_arrivals += 1
            service_start_time = buffet_system.env.now
            service_time = random.expovariate(self.service_rate)
            self.service_times.append(service_time)
            yield buffet_system.env.timeout(service_time)
            service_end_time = buffet_system.env.now
            wait_time = service_start_time - arrival_time
            self.waiting_times.append(wait_time)
            self.response_time.append(service_end_time-arrival_time)
            self.number_of_completions.append([self.number_of_completions[-1][0]+1,buffet_system.env.now])
            # calculate next destination
            next_destination = np.random.choice(len(self.leave_list), p=self.leave_list)
            if next_destination == 0:
                # go to Seafood
                buffet_system.env.process(buffet_system.seafoodQueue.customer_arrival(buffet_system, customer_id))

            elif next_destination == 1:
                # go out
                buffet_system.current_job.append(buffet_system.current_job[len(buffet_system.current_job)-1] - 1)
                buffet_system.completed_job += 1
            else:
                # go to Dessert
                buffet_system.env.process(buffet_system.dessertQueue.customer_arrival(buffet_system, customer_id))

class DessertQueue(MMNQueue):
    def customer_arrival(self, buffet_server, customer_id):
        arrival_time = buffet_server.env.now
        with self.server.request() as request:
            yield request
            self.total_arrivals += 1
            service_start_time = buffet_server.env.now
            service_time = random.expovariate(self.service_rate)
            self.service_times.append(service_time)
            yield buffet_server.env.timeout(random.expovariate(self.service_rate))
            service_end_time = buffet_server.env.now
            wait_time = service_start_time - arrival_time
            self.waiting_times.append(wait_time)
            self.response_time.append(service_end_time-arrival_time)
            self.number_of_completions.append([self.number_of_completions[-1][0]+1,buffet_server.env.now])
            # calculate next destination
            next_destination = np.random.choice(len(self.leave_list), p=self.leave_list)
            if next_destination == 0:
                # go to Seafood
                buffet_server.env.process(buffet_server.seafoodQueue.customer_arrival(buffet_server, customer_id))
            elif next_destination == 1:
                # go to soup
                buffet_server.env.process(buffet_server.soupQueue.customer_arrival(buffet_server, customer_id))
            else:
                # go out
                buffet_server.current_job.append(buffet_server.current_job[len(buffet_server.current_job)-1] - 1)
                buffet_server.completed_job += 1

class PlotMetric():
    def __init__(self, waiting_times, completion_list):
        self.waiting_times = waiting_times
        self.completion_list = completion_list
    def plotAverage(self, target, xlabel, ylabel, title, accumulation):
        average = [sum(target[:i]) / len(target[:i]) for i in range(1, len(target), accumulation)]
        time_points1 = list(range(1, len(average)+1))
        # Plotting the average waiting time over time

        plt.plot(time_points1, average, marker='o', linestyle='-', color='green')

        # Adding labels and title
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        # Display the plot
        plt.show()
    def plotTarget(self, target, xlabel, ylabel, title, accumulation):

        time_points1 = list(range(1, len(target)+1))
        # Plotting the average waiting time over time

        plt.plot(time_points1, target, marker='o', linestyle='-', color='green')

        # Adding labels and title
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        xlim = plt.xlim()
        f = 1
        plt.xlim((xlim[0] * f, xlim[1] * f))
        # Display the plot
        plt.show()
    def plotThroughput(target, xlabel, ylabel, title, accumulation=300):


        throughput = [ round(target[i][0] / target[i][1],2)  for i in range(1,len(target))]
        throughput.insert(0,0);
        # averageThroughput = ([sum(throughput[:i]) / len(throughput[:i]) for i in range(50, len(throughput), accumulation)])

        # print(averageThroughput)
        timeplot = [target[i][1] for i in range(0, len(throughput))]

        plt.plot(timeplot , throughput, marker='o', linestyle='-', color='green')

        # Adding labels and title
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        xlim = plt.xlim()
        f = 1
        plt.xlim((xlim[0] * f, xlim[1] * f))
        # Display the plot
        plt.show()

    def plotNonLinearRegression(self, target, xlabel, ylabel, title):
        def nonlinear_function(x, a, b, c):
            # Định nghĩa hàm non-linear, ở đây chọn là hàm ax^2 + bx + c
            return a * x**2 + b * x + c

        # Chia dữ liệu thành hai mảng riêng lẻ cho x và y
        x_data = [point[1] for point in target]
        y_data = [point[0] / point[1] if point[1] != 0 else 0 for point in target]

        # Thực hiện non-linear fit
        popt, _ = curve_fit(nonlinear_function, x_data, y_data)

        # Tạo một mảng x mới cho đồ thị với độ tăng tự do
        x_fit = np.linspace(min(x_data), max(x_data), 100)

        # Tính giá trị y dựa trên non-linear regression
        y_fit = nonlinear_function(x_fit, *popt)

        # Vẽ đồ thị non-linear regression mà không vẽ các điểm chấm
        plt.plot(x_fit, y_fit, '--', label='Non-Linear Regression')

        # Thêm tiêu đề và nhãn cho đồ thị
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        plt.legend()

        # Hiển thị đồ thị
        plt.show()

class SanityTest():
    def seafood_queue():
        # Set leave_matrix values for seafoodQueue
        leave_matrix = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

        # Set arrival_matrix values for seafoodQueue
        arrival_matrix = [1, 0, 0]

        random_seed = 39
        arrival_rate = 3
        service_rate = np.array([7, 7, 7])
        num_servers = np.array([2, 2, 2])
        num_customers = 10000

        env = simpy.Environment()
        buffetSystem = BuffetSystem(env, arrival_rate, service_rate, num_servers, arrival_matrix, leave_matrix)
        env.process(buffetSystem.run(num_customers))
        env.run()
        buffetSystem.seafoodQueue.end_simulation_time = env.now
        buffetSystem.seafoodQueue.calculate_metrics()
        # Check calculation result
        assert buffetSystem.seafoodQueue.calculation_metrics[0] == 0.15, "Wrong response_time calculation"
        assert buffetSystem.seafoodQueue.calculation_metrics[1] == 3, "Wrong throughput calculation"
        assert buffetSystem.seafoodQueue.calculation_metrics[2] == 0.01, "Wrong average_waiting_time calculation"
        assert buffetSystem.seafoodQueue.calculation_metrics[3] == 0.21, "Wrong utilization calculation"
        assert buffetSystem.seafoodQueue.calculation_metrics[4] == 0.14, "Wrong service_time calculation"
        assert buffetSystem.seafoodQueue.calculation_metrics[5] == 0.33, "Wrong interarrival_time calculation"
        # Check if the difference between simulation and calculation metrics is within 5%
        for sim_metric, calc_metric in zip(buffetSystem.seafoodQueue.simulation_metrics, buffetSystem.seafoodQueue.calculation_metrics):
            if isinstance(sim_metric, float) and isinstance(calc_metric, float) and calc_metric != 0:
                assert abs(sim_metric - calc_metric) / calc_metric <= 0.05, "Difference in metrics exceeds 5%."
            elif sim_metric != "null" and calc_metric == "null":
                assert False, "Calculation metric is 'null' while simulation metric is not."
        print("seafood: Passed Sanity Test")
    def soup_queue():
        # Set leave_matrix values for soupQueue
        leave_matrix = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

        # Set arrival_matrix values for soupQueue
        arrival_matrix = [0, 1, 0]

        random_seed = 39
        arrival_rate = 3
        service_rate = np.array([7, 7, 7])
        num_servers = np.array([2, 2, 2])
        num_customers = 10000

        env = simpy.Environment()
        buffetSystem = BuffetSystem(env, arrival_rate, service_rate, num_servers, arrival_matrix, leave_matrix)
        env.process(buffetSystem.run(num_customers))
        env.run()
        buffetSystem.soupQueue.end_simulation_time = env.now
        buffetSystem.soupQueue.calculate_metrics()
        # Check calculation result
        assert buffetSystem.soupQueue.calculation_metrics[0] == 0.15, "Wrong response_time calculation"
        assert buffetSystem.soupQueue.calculation_metrics[1] == 3, "Wrong throughput calculation"
        assert buffetSystem.soupQueue.calculation_metrics[2] == 0.01, "Wrong average_waiting_time calculation"
        assert buffetSystem.soupQueue.calculation_metrics[3] == 0.21, "Wrong utilization calculation"
        assert buffetSystem.soupQueue.calculation_metrics[4] == 0.14, "Wrong service_time calculation"
        assert buffetSystem.soupQueue.calculation_metrics[5] == 0.33, "Wrong interarrival_time calculation"
        # Check if the difference between simulation and calculation metrics is within 5%
        for sim_metric, calc_metric in zip(buffetSystem.soupQueue.simulation_metrics, buffetSystem.soupQueue.calculation_metrics):
            if isinstance(sim_metric, float) and isinstance(calc_metric, float) and calc_metric != 0:
                assert abs(sim_metric - calc_metric) / calc_metric <= 0.05, "Difference in metrics exceeds 5%."
            elif sim_metric != "null" and calc_metric == "null":
                assert False, "Calculation metric is 'null' while simulation metric is not."
        print("soup: Passed Sanity Test")
    def dessert_queue():
        # Set leave_matrix values for DessertQueue
        leave_matrix = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

        # Set arrival_matrix values for DessertQueue
        arrival_matrix = [0, 0, 1]
        random_seed = 39
        arrival_rate = 3
        service_rate = np.array([7, 7, 7])
        num_servers = np.array([2, 2, 2])
        num_customers = 10000

        env = simpy.Environment()
        buffetSystem = BuffetSystem(env, arrival_rate, service_rate, num_servers, arrival_matrix, leave_matrix)
        env.process(buffetSystem.run(num_customers))
        env.run()
        buffetSystem.dessertQueue.end_simulation_time = env.now
        buffetSystem.dessertQueue.calculate_metrics()
        # Check calculation result
        assert buffetSystem.dessertQueue.calculation_metrics[0] == 0.15, "Wrong response_time calculation"
        assert buffetSystem.dessertQueue.calculation_metrics[1] == 3, "Wrong throughput calculation"
        assert buffetSystem.dessertQueue.calculation_metrics[2] == 0.01, "Wrong average_waiting_time calculation"
        assert buffetSystem.dessertQueue.calculation_metrics[3] == 0.21, "Wrong utilization calculation"
        assert buffetSystem.dessertQueue.calculation_metrics[4] == 0.14, "Wrong service_time calculation"
        assert buffetSystem.dessertQueue.calculation_metrics[5] == 0.33, "Wrong interarrival_time calculation"
        # Check if the difference between simulation and calculation metrics is within 5%
        for sim_metric, calc_metric in zip(buffetSystem.dessertQueue.simulation_metrics, buffetSystem.dessertQueue.calculation_metrics):
            if isinstance(sim_metric, float) and isinstance(calc_metric, float) and calc_metric != 0:
                assert abs(sim_metric - calc_metric) / calc_metric <= 0.1, "Difference in metrics exceeds 10%."
            elif sim_metric != "null" and calc_metric == "null":
                assert False, "Calculation metric is 'null' while simulation metric is not."
        print("dessert: Passed Sanity Test")
    def test_all_queue(self):
        self.seafood_queue()
        self.soup_queue()
        self.dessert_queue()
def main():
    random_seed = 39
    random.seed( random_seed )
    arrival_rate =  2 # Arrival rate (customers per time unit)
    service_rate = np.array([5, 6, 7])  # Service rate (customers per time unit per server)
    num_servers = np.array([2, 1, 5])  # Number of servers
    num_customers = 100000  # Number of customers to simulate
    arrival_matrix = np.array([0.45, 0.4, 0.15])
    leave_matrix = np.array([[0, 0.3, 0.2],
                             [0.2, 0, 0.2],
                             [0.1, 0.1, 0]])
    # arrival_rate = 0.2  # Arrival rate (customers per time unit)
    # service_rate = np.array([1/6, 6, 6])  # Service rate (customers per time unit per server)
    # num_servers = np.array([5, 0, 0])  # Number of servers
    # num_customers = 10000  # Number of customers to simulate
    # arrival_matrix = np.array([1, 0, 0])
    # leave_matrix = np.array([[0, 0, 0],
    #                          [0, 0, 0],
    #                          [0, 0, 0]])

    env = simpy.Environment()
    buffetSystem = BuffetSystem(env, arrival_rate, service_rate, num_servers, arrival_matrix, leave_matrix)
    env.process(buffetSystem.run(num_customers))
    env.run()
    buffetSystem.display_metric()
if __name__ == "__main__":
    SanityTest.test_all_queue(SanityTest)
    main()

seafood: Passed Sanity Test
soup: Passed Sanity Test
dessert: Passed Sanity Test


In [ ]:
# @title Hidden
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define a quadratic function
def quadratic_function(x, a, b, c):
    return a * x**2 + b * x + c

# Generate some example data
np.random.seed(42)
x = 2 * np.random.rand(100, 1)
y = 2 * x**2 + 3 * x + 1 + np.random.randn(100, 1)

# Fit the quadratic function to the data
params, covariance = curve_fit(quadratic_function, x.flatten(), y.flatten())

# Generate points for the fitted curve
x_fit = np.linspace(0, 2, 100)
y_fit = quadratic_function(x_fit, *params)

# Plot the original data points
plt.scatter(x, y, color='blue')

# Plot the fitted curve
plt.plot(x_fit, y_fit, color='red', label='Fitted Curve')

# Show the plot
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Non-linear Regression Example')
plt.legend()
plt.show()
